# Init

In [2]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical
from keras import optimizers, metrics
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import LabelEncoder
import copy

In [3]:
def loadPreapreData(datafile):
    dataset = pd.read_csv(datafile)
    X = dataset.iloc[:, :-1].values
    y = dataset.iloc[:, -1].values

    encoder = LabelEncoder()
    encoder.fit(y)
    y_enc = encoder.transform(y)
    y_cat = to_categorical(y_enc)

    return (X, y_cat, X[0].size, y_cat[0].size)

In [4]:
def getModel(in_dim, out_dim, optimizer, metrics):
    model = Sequential()
    model.add(Dense(10, input_dim=in_dim, activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(out_dim, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=metrics)
    return model


In [5]:
def fitWithKfold(X, y ,model, epochs):
    kfold = KFold(n_splits=10, shuffle=True, random_state=42)

    scores = []
    for train, test in kfold.split(X):
        model.fit(X[train], y[train], epochs=epochs, batch_size=10, verbose=0)
        score = model.evaluate(X[test], y[test], verbose=0)
        scores.append(score[1] * 100)

    # Wyświetlenie wyników walidacji krzyżowej K-fold
    print('Score: %.2f%% (+/- %.2f%%)' % (np.mean(scores), np.std(scores)))

In [6]:
optimizers_dict = {
    'SGD': optimizers.SGD(),
    'RMSprop': optimizers.RMSprop(),
    'Adam': optimizers.Adam(),
    'Adadelta': optimizers.Adadelta(),
    'Adagrad': optimizers.Adagrad(),
    'Adamax': optimizers.Adamax(),
    'Nadam': optimizers.Nadam(),
    'Ftrl': optimizers.Ftrl()
}

metrics_dict = {
    'MeanSquaredError': metrics.MeanSquaredError(),
    'BinaryAccuracy': metrics.BinaryAccuracy(),
    'CategoricalAccuracy': metrics.CategoricalAccuracy(),
    # 'SensitivityAtSpecificity': metrics.SensitivityAtSpecificity(),
    # 'SpecificityAtSensitivity': metrics.SpecificityAtSensitivity(),
    'AUC': metrics.AUC(),
    'Precision': metrics.Precision(),
    'Recall': metrics.Recall(),
    'FalsePositives': metrics.FalsePositives(),
    'FalseNegatives': metrics.FalseNegatives(),
    'TruePositives': metrics.TruePositives(),
    'TrueNegatives': metrics.TrueNegatives()
}


# Iris

In [10]:
X, y, in_dim, out_dim = loadPreapreData("iris.data")
for oprimizer_name,oprimizer in optimizers_dict.items():
    for metric_name,metric in metrics_dict.items():
        model = getModel(in_dim,out_dim,oprimizer,metric)
        print("Iris classyfication model for optimizer: "+ oprimizer_name + ", metric: "+ metric_name)
        fitWithKfold(X, y ,model, 50)

Iris classyfication model for optimizer: SGD, metric: MeanSquaredError
Score: 2.06% (+/- 2.84%)
Iris classyfication model for optimizer: SGD, metric: BinaryAccuracy
Score: 98.19% (+/- 2.22%)
Iris classyfication model for optimizer: SGD, metric: CategoricalAccuracy
Score: 95.29% (+/- 4.29%)
Iris classyfication model for optimizer: SGD, metric: AUC
Score: 99.62% (+/- 0.86%)
Iris classyfication model for optimizer: SGD, metric: Precision
Score: 93.95% (+/- 5.55%)
Iris classyfication model for optimizer: SGD, metric: Recall


KeyboardInterrupt: 

# Cars

In [29]:
def loadPreapreData_Cars(datafile):
    dataset = pd.read_csv(datafile)
    X = dataset.iloc[:, :-1].values
    y = dataset.iloc[:, -1].values
    X_enc=copy.deepcopy(X)

    for i in range(X[0].size):
        encoder = LabelEncoder()
        encoder.fit(X[:,i])
        X_enc[:,i] = encoder.transform(X[:,i]).astype("float")
        # X_enc[:,i] = X_enc[:,i]/max(X_enc[:,i])

    encoder = LabelEncoder()
    encoder.fit(y)
    y_enc = encoder.transform(y)

    y_cat = to_categorical(y_enc)

    return (X_enc, y_cat, X_enc[0].size, y_cat[0].size)

In [30]:
X, y, in_dim, out_dim = loadPreapreData_Cars("car.data")
print(X)
model = getModel(in_dim,out_dim,"adam",["accuracy"])
fitWithKfold(X, y ,model, 50)

[[3.0 3.0 0.0 0.0 2.0 2.0]
 [3.0 3.0 0.0 0.0 2.0 0.0]
 [3.0 3.0 0.0 0.0 1.0 1.0]
 ...
 [1.0 1.0 3.0 2.0 0.0 1.0]
 [1.0 1.0 3.0 2.0 0.0 2.0]
 [1.0 1.0 3.0 2.0 0.0 0.0]]


ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type float).

# Wine

In [42]:
def loadPreapreData_Wine(datafile):
    dataset = pd.read_csv(datafile,delimiter=";",header=1)
    X = dataset.iloc[:, :-1].values
    y = dataset.iloc[:, -1].values

    encoder = LabelEncoder()
    encoder.fit(y)
    y_enc = encoder.transform(y)
    y_cat = to_categorical(y_enc)

    return (X, y_cat, X[0].size, y_cat[0].size)

In [43]:
X, y, in_dim, out_dim = loadPreapreData_Wine("winequality-red.csv")
for oprimizer_name,oprimizer in optimizers_dict.items():
    for metric_name,metric in metrics_dict.items():
        model = getModel(in_dim,out_dim,oprimizer,metric)
        print("Wine quailty model for optimizer: "+ oprimizer_name + ", metric: "+ metric_name)
        fitWithKfold(X, y ,model, 20)

[[0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 ...
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]]
Wine quailty model for optimizer: SGD, metric: MeanSquaredError
Score: 9.86% (+/- 0.60%)
Wine quailty model for optimizer: SGD, metric: BinaryAccuracy
Score: 84.51% (+/- 0.97%)
Wine quailty model for optimizer: SGD, metric: CategoricalAccuracy


KeyboardInterrupt: 